In [3]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne

import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

# TODO: hmm fix these utils imports, import the funcs individually. 6/1/25.
from src.analysis.utils.general_utils import *
from src.analysis.utils.general_utils import make_or_load_subjects_electrodes_to_ROIs_dict
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup

from src.analysis.config import experiment_conditions

from src.analysis.utils.labeled_array_utils import (
    put_data_in_labeled_array_per_roi_subject,
    remove_nans_from_labeled_array,
    remove_nans_from_all_roi_labeled_arrays,
    concatenate_conditions_by_string,
    get_data_in_time_range
)

from src.analysis.decoding.decoding import (
    concatenate_and_balance_data_for_decoding, 
    get_and_plot_confusion_matrix_for_rois_jim,
    Decoder, 
    windower,
    get_confusion_matrices_for_rois_time_window_decoding_jim,
    compute_accuracies,
    perform_time_perm_cluster_test_for_accuracies,
    plot_accuracies
)

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

In [4]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
explained_variance = 0.8
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [5]:
task='GlobalLocal'
conditions = experiment_conditions.stimulus_congruency_conditions # set this to whichever conditions you're running
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_power_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline corr

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:422: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (198, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 119.8
    Max NaN count in a trial: 1282
    Stimulus_c: 198 valid trials out of 198
  Loading condition: Stimulus_i with parameters: {'BIDS_events': ['Stimulus/i25.0/Accuracy1.0', 'Stimulus/i75.0/Accuracy1.0'], 'congruency': 'i'}
Not setting metadata
207 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:422: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (207, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 123.9
    Max NaN count in a trial: 1923
    Stimulus_i: 207 valid trials out of 207
  Loading condition: Stimulus_c with parameters: {'BIDS_events': ['Stimulus/c25.0/Accuracy1.0', 'Stimulus/c75.0/Accuracy1.0'], 'congruency': 'c'}
Not setting metadata
198 matching events found
No baseline correction applied


...

    Original shape: (194, 204, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 948.3
    Max NaN count in a trial: 19230
    Stimulus_i: 194 valid trials out of 194
  Loading condition: Stimulus_c with parameters: {'BIDS_events': ['Stimulus/c25.0/Accuracy1.0', 'Stimulus/c75.0/Accuracy1.0'], 'congruency': 'c'}
Not setting metadata
206 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:422: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (206, 204, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 1288.2
    Max NaN count in a trial: 42947
    Stimulus_c: 206 valid trials out of 206
  Loading condition: Stimulus_i with parameters: {'BIDS_events': ['Stimulus/i25.0/Accuracy1.0', 'Stimulus/i75.0/Accuracy1.0'], 'congruency': 'i'}
Not setting metadata
194 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:422: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (194, 204, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 948.3
    Max NaN count in a trial: 19230
    Stimulus_i: 194 valid trials out of 194


In [6]:
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus_c', 'Stimulus_i']

get significant channels

In [7]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103
Loaded significant channels for subject D0107A
Loaded significant channels for subject D0110
Loaded significant channels for subject D0116
Loaded significant channels for subject D0117
Loaded significant channels for subject D0121


In [8]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
all_electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LAS

let's manually make the occ filtered sig electrodes

In [9]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

all_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [10]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

all_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [11]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [12]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, all_electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 265
Total number of significant lpfc electrodes across all subjects: 44
Total number of v1 electrodes across all subjects: 18
Total number of significant v1 electrodes across all subjects: 6
Total number of occ electrodes across all subjects: 73
Total number of significant occ electrodes across all subjects: 18
Total number of occ_filtered electrodes across all subjects: 23
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 10
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [13]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.
Subject D0107A has the expected sampling rate: 256.0 Hz.
Subject D0110 has the expected sampling rate: 256.0 Hz.
Subject D0116 has the expected sampling rate: 256.0 Hz.
Subject D0117 has the expected sampling rate: 256.0 Hz.
Subject D0121 has the expected sampling rate: 256.0 Hz.


make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.


In [14]:
# Example usage:

# choose whether to use sig elecs or all elecs
electrodes = all_electrodes_per_subject_roi # toggle this to sig_electrodes_per_subject_roi if just using sig elecs, or electrodes_per_subject_roi if using all elecs

if electrodes == all_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'all_elecs'
elif electrodes == sig_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'sig_elecs'
else:
    elec_string_to_add_to_filename = None

roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(
    subjects_mne_objects,
    condition_names,
    rois,
    subjects,
    electrodes, 
    obs_axs=0,  # Trials dimension (ignoring the conditions dimension for now cuz we iterate over it)
    chans_axs=1,  # Channels dimension
    time_axs=2,   # Time dimension
    random_state=42  # For reproducibility
)

ROI 'lpfc': Maximum trials per condition:
  Condition 'Stimulus_c': 222 trials from subjects ['D0116']
  Condition 'Stimulus_i': 219 trials from subjects ['D0110']
in roi lpfc, subject D0057 has 198 trials for condition Stimulus_c
in roi lpfc, subject D0057 has 207 trials for condition Stimulus_i
Subject D0057, ROI lpfc, LabeledArray shape: (2, 222, 17, 641)
in roi lpfc, subject D0059 has 216 trials for condition Stimulus_c
in roi lpfc, subject D0059 has 211 trials for condition Stimulus_i
Subject D0059, ROI lpfc, LabeledArray shape: (2, 222, 5, 641)
in roi lpfc, subject D0063 has 221 trials for condition Stimulus_c
in roi lpfc, subject D0063 has 168 trials for condition Stimulus_i
Subject D0063, ROI lpfc, LabeledArray shape: (2, 222, 36, 641)
in roi lpfc, subject D0065 has 177 trials for condition Stimulus_c
in roi lpfc, subject D0065 has 139 trials for condition Stimulus_i
Subject D0065, ROI lpfc, LabeledArray shape: (2, 222, 9, 641)
in roi lpfc, subject D0069 has 177 trials for cond

remove trials with nans from roi labeled arrays if just getting the minimum number of trials for all subjects

In [15]:
roi_labeled_arrays_no_nans, conditions_with_no_valid_trials_per_roi = remove_nans_from_all_roi_labeled_arrays(roi_labeled_arrays, obs_axs=0, chans_axs=1, time_axs=2)

set up decoding output directory and conditions to compare 9/12

In [16]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'decoding', 'figs', f"{epochs_root_file}")
os.makedirs(save_dir, exist_ok=True)

print(f"Save directory created or already exists at: {save_dir}")

condition_comparisons = {}

if conditions == experiment_conditions.stimulus_experiment_conditions:
    condition_comparisons['congruency'] = [['c25', 'c75'], ['i25', 'i75']]
    condition_comparisons['switchType'] = [['r25', 'r75'], ['s25', 's75']]
    
elif conditions == experiment_conditions.stimulus_conditions:
    condition_comparisons['BigLetter'] = ['bigS', 'bigH']
    condition_comparisons['SmallLetter'] = ['smallS', 'smallH']
    condition_comparisons['Task'] = ['taskG', 'taskL']

elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    condition_comparisons['BigLetter'] = ['bigS', 'bigH']
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    condition_comparisons['SmallLetter'] = ['smallS', 'smallH']
elif conditions == experiment_conditions.stimulus_task_conditions:
    condition_comparisons['task'] = ['taskG', 'taskL']

elif conditions == experiment_conditions.stimulus_congruency_conditions:
    condition_comparisons['congruency'] = [['Stimulus_c'], ['Stimulus_i']]
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    condition_comparisons['switchType'] = [['Stimulus_r'], ['Stimulus_s']]

Save directory created or already exists at: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\decoding\figs\Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False


In [17]:
# get the confusion matrix using the downsampled version

# add elec and subject info to filename 6/11/25
other_string_to_add = elec_string_to_add_to_filename + '_' + str(len(subjects)) + '_subjects'

for condition_comparison, strings_to_find in condition_comparisons.items():
    confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
        roi_labeled_arrays=roi_labeled_arrays_no_nans,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        other_string_to_add=elec_string_to_add_to_filename,
        n_splits=5,
        n_repeats=10,
        obs_axs=0,
        balance_method='subsample',  # Use 'subsample' to balance by subsampling
        explained_variance=explained_variance,
        random_state=42  # For reproducibility
    )

Processing ROI: lpfc

ROI: lpfc
Initial concatenated data shape: (132, 265, 641)
  Trials: 132
  Channels: 265
  Time points: 641

--- Detailed NaN Analysis ---

Missing Channels (all NaN across all trials): 0/265

Trial-level NaN Statistics:
  Trials with ANY NaN: 30/132 (22.7%)
  Trials with outlier NaNs (sparse): 0/132 (0.0%)
  Trials with missing channel NaNs (dense): 30/132 (22.7%)
  Trials with both types: 0/132 (0.0%)

Missing Channel Statistics (for affected trials):
  Mean missing channels per affected trial: 265.0
  Max missing channels in a trial: 265

Overall NaN Percentage: 22.73%

--- NaN Impact by Condition ---
  ['Stimulus_c']: 0/66 trials with NaNs (0.0%)
  ['Stimulus_i']: 30/66 trials with NaNs (45.5%)

After removing NaN trials:
  Trials kept: 102/132 (77.3%)
  New data shape: (102, 265, 641)

--- Final Trial Counts by Condition ---
  Condition ['Stimulus_c']: 66 trials
  Condition ['Stimulus_i']: 36 trials

Subsampling to min trial count: 36
  Subsampled condition 0

In [ ]:
# get the confusion matrix using the version with nan trials and doing nan trial mixup
for condition_comparison, strings_to_find in condition_comparisons.items():
    confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
        roi_labeled_arrays=roi_labeled_arrays,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        n_splits=5,
        n_repeats=10,
        obs_axs=0,
        balance_method='pad_with_nans',  # Use 'pad_with_nans' to balance by padding with nans,
        explained_variance=explained_variance,
        random_state=42  # For reproducibility
    )


prestimulus vs poststimulus confusion matrices 9/21

In [ ]:
# # Define the time intervals of interest
# pre_stimulus_interval = (-1.0, 0.0)  # Pre-stimulus: time -1 to 0 seconds
# post_stimulus_interval = (0.0, 1.5)  # Post-stimulus: time 0 to 1.5 seconds
# pre_stimulus_roi_labeled_arrays = {}
# post_stimulus_roi_labeled_arrays = {}

# for roi, labeled_array in roi_labeled_arrays.items():
#     pre_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, pre_stimulus_interval)
#     post_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, post_stimulus_interval)


# pre_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
#     roi_labeled_arrays=pre_stimulus_roi_labeled_arrays,
#     rois=rois,
#     condition_comparison=condition_comparison,
#     strings_to_find=strings_to_find,
#     save_dir=save_dir,
#     time_interval_name='pre_stimulus',  # If not dealing with specific time intervals
#     n_splits=5,
#     n_repeats=10,
#     obs_axs=0,
#     balance_method='subsample',  # Use 'subsample' to balance by subsampling
#     random_state=42  # For reproducibility
# )

# post_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
#     roi_labeled_arrays=post_stimulus_roi_labeled_arrays,
#     rois=rois,
#     condition_comparison=condition_comparison,
#     strings_to_find=strings_to_find,
#     save_dir=save_dir,
#     time_interval_name='post_stimulus',  # If not dealing with specific time intervals
#     n_splits=5,
#     n_repeats=10,
#     obs_axs=0,
#     balance_method='subsample',  # Use 'subsample' to balance by subsampling
#     random_state=42  # For reproducibility
# )

define colors for plotting (not used yet as of 8/21)

In [ ]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}

new attempt at time window sliding decoding with significance 11/23  

actually do the time window decoding, run stats, and plot accuracies

In [18]:
# Main code

# Directory to save confusion matrices
cm_save_dir = os.path.join(save_dir, "confusion_matrices")
os.makedirs(cm_save_dir, exist_ok=True)

condition_comparison_confusion_matrices = {}
n_splits=5
n_repeats=10
balance_method='subsample'
random_state=42
window_size=64
step_size=32
n_permutations=200

for condition_comparison, strings_to_find in condition_comparisons.items():
    # Get confusion matrices for each ROI
    cm_true_per_roi, cm_shuffle_per_roi = get_confusion_matrices_for_rois_time_window_decoding_jim(
        roi_labeled_arrays=roi_labeled_arrays,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        n_splits=n_splits,
        n_repeats=n_repeats,
        obs_axs=0,
        time_axs=-1,
        balance_method=balance_method,
        explained_variance=explained_variance,
        random_state=random_state,
        window_size=window_size,
        step_size=step_size,
        n_permutations=n_permutations,
        sampling_rate=sampling_rate,
        first_time_point=-1
    )

    np.save(os.path.join(cm_save_dir, f'{condition_comparison}_{n_splits}_splits_{n_repeats}_repeats_{balance_method}_balance_method_{random_state}_random_state_{window_size}_window_size_{step_size}_step_size_{n_permutations}_permutations_{sampling_rate}_sampling_rate_cm_true_per_roi.npy'), cm_true_per_roi)
    np.save(os.path.join(cm_save_dir, f'{condition_comparison}_{n_splits}_splits_{n_repeats}_repeats_{balance_method}_balance_method_{random_state}_random_state_{window_size}_window_size_{step_size}_step_size_{n_permutations}_permutations_{sampling_rate}_sampling_rate_cm_shuffle_per_roi.npy'), cm_shuffle_per_roi)

    # Store the results in a dictionary
    condition_comparison_confusion_matrices[condition_comparison] = {
        'strings_to_find': strings_to_find,
        'cm_true_per_roi': cm_true_per_roi,
        'cm_shuffle_per_roi': cm_shuffle_per_roi
    }

    # Now compute accuracies and perform time permutation cluster test
    for roi in rois:
        cm_true = cm_true_per_roi[roi]['cm_true']
        cm_shuffle = cm_shuffle_per_roi[roi]['cm_shuffle']
        time_window_centers = cm_true_per_roi[roi]['time_window_centers']
        window_size = cm_true_per_roi[roi]['window_size']
        step_size = cm_true_per_roi[roi]['step_size']

        # Compute accuracies
        accuracies_true, accuracies_shuffle = compute_accuracies(cm_true, cm_shuffle)

        # Perform time permutation cluster test
        significant_clusters, p_values = perform_time_perm_cluster_test_for_accuracies(
            accuracies_true, accuracies_shuffle, p_thresh=0.05, n_perm=100, seed=42
        )

        # Store significant clusters and p-values
        cm_true_per_roi[roi]['significant_clusters'] = significant_clusters
        cm_true_per_roi[roi]['p_values'] = p_values

        # Optionally, store accuracies
        cm_true_per_roi[roi]['accuracies_true'] = accuracies_true
        cm_shuffle_per_roi[roi]['accuracies_shuffle'] = accuracies_shuffle
        print(significant_clusters)

        # Plot accuracies
        plot_accuracies(
            time_points=time_window_centers,
            accuracies_true=accuracies_true,
            accuracies_shuffle=accuracies_shuffle,
            significant_clusters=significant_clusters,
            window_size=window_size,
            step_size=step_size,
            sampling_rate=sampling_rate,
            condition_comparison=condition_comparison,
            roi=roi,
            save_dir=save_dir
        )

Processing ROI: lpfc

ROI: lpfc
Initial concatenated data shape: (444, 265, 641)
  Trials: 444
  Channels: 265
  Time points: 641

--- Detailed NaN Analysis ---

Missing Channels (all NaN across all trials): 0/265

Trial-level NaN Statistics:
  Trials with ANY NaN: 342/444 (77.0%)
  Trials with outlier NaNs (sparse): 0/444 (0.0%)
  Trials with missing channel NaNs (dense): 342/444 (77.0%)
  Trials with both types: 0/444 (0.0%)

Missing Channel Statistics (for affected trials):
  Mean missing channels per affected trial: 44.9
  Max missing channels in a trial: 265

Overall NaN Percentage: 13.04%

--- NaN Impact by Condition ---
  ['Stimulus_c']: 156/222 trials with NaNs (70.3%)
  ['Stimulus_i']: 186/222 trials with NaNs (83.8%)

After removing NaN trials:
  Trials kept: 102/444 (23.0%)
  New data shape: (102, 265, 641)

--- Final Trial Counts by Condition ---
  Condition ['Stimulus_c']: 66 trials
  Condition ['Stimulus_i']: 36 trials

Subsampling to min trial count: 36
  Subsampled cond

NameError: name 'Union' is not defined

In [ ]:
for condition_comparison in condition_comparison_confusion_matrices:
    cm_true_per_roi = condition_comparison_confusion_matrices[condition_comparison]['cm_true_per_roi']
    for roi in cm_true_per_roi:
        significant_clusters = cm_true_per_roi[roi]['significant_clusters']
        print(f'significant clusters for {condition_comparison} and {roi} are: {significant_clusters}')

        p_values = cm_true_per_roi[roi]['p_values']
        print(f'p values for {condition_comparison} and {roi} are: {p_values}')

        accuracies_true = cm_true_per_roi[roi]['accuracies_true']
        print(f'accuracies for {condition_comparison} and {roi} are: {accuracies_true}')

        # Now you can use significant_clusters, p_values, and accuracies_true as needed

testing code